In [1]:
# 💾 Guardado de chunks en archivos JSONL divididos por partes
import json
import os
import numpy as np
import faiss
import pickle

# 📦 Carga de .jsonl línea por línea usando JSONLoader (modo json_lines)
from langchain_community.document_loaders import JSONLoader

# ✂️ Definición de splitters (Character y Recursive)
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

# 🧠 Cargar modelo de embeddings (sentence-transformers)
from sentence_transformers import SentenceTransformer


from langchain.schema import Document


from langchain.vectorstores.faiss import FAISS

from langchain.embeddings import HuggingFaceEmbeddings

from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
import textwrap


In [10]:
# Cargamos cada línea del archivo como un JSON independiente (modo json_lines=True)
# Extraemos solo el campo "document_text" de cada línea
# Devolvemos objetos Document(page_content=..., metadata={})

loader = JSONLoader(
    file_path="./Input/all_drugs_docs.jsonl",
    jq_schema=".document_text",       # Extrae solo el contenido del texto
    text_content=False,               # Nos devuelve Document(), no solo string
    json_lines=True                   # 💡 ¡IMPORTANTE! Activa modo .jsonl (una línea = un JSON)
)

documents = loader.load()
print(f"📄 Documentos cargados correctamente: {len(documents)}")


KeyboardInterrupt: 

In [ ]:
# Configuración global del chunking
chunk_size = 1000      # Tamaño de cada fragmento (en caracteres)
chunk_overlap = 100    # Superposición entre chunks (para no cortar ideas)

# Splitter 1: Corta por bloques de caracteres fijos
char_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

# Splitter 2: Intenta cortar primero por saltos de línea, luego por frases, luego por palabras
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

print("✅ Splitters configurados correctamente.")


✅ Splitters configurados correctamente.


In [24]:
# 🔁 Aplicación de los splitters

# Aplicamos el CharacterTextSplitter a todos los documentos
char_chunks = char_splitter.split_documents(documents)
print(f"🔹 CharacterTextSplitter generó {len(char_chunks)} chunks")

# Aplicamos el RecursiveCharacterTextSplitter a todos los documentos
recursive_chunks = recursive_splitter.split_documents(documents)
print(f"🔸 RecursiveCharacterTextSplitter generó {len(recursive_chunks)} chunks")

Created a chunk of size 2313, which is longer than the specified 1000
Created a chunk of size 1286, which is longer than the specified 1000
Created a chunk of size 1731, which is longer than the specified 1000
Created a chunk of size 1217, which is longer than the specified 1000
Created a chunk of size 1089, which is longer than the specified 1000
Created a chunk of size 2376, which is longer than the specified 1000
Created a chunk of size 1574, which is longer than the specified 1000
Created a chunk of size 2547, which is longer than the specified 1000
Created a chunk of size 1987, which is longer than the specified 1000
Created a chunk of size 3582, which is longer than the specified 1000
Created a chunk of size 1847, which is longer than the specified 1000
Created a chunk of size 9711, which is longer than the specified 1000
Created a chunk of size 2317, which is longer than the specified 1000
Created a chunk of size 1689, which is longer than the specified 1000
Created a chunk of s

🔹 CharacterTextSplitter generó 221652 chunks
🔸 RecursiveCharacterTextSplitter generó 452732 chunks


In [25]:
# Crear carpeta de salida si no existe
os.makedirs("./Output", exist_ok=True)

# Función para guardar chunks divididos en partes
def save_chunks_in_parts(chunks, name_prefix, part_size=50000):
    for i in range(0, len(chunks), part_size):
        part = chunks[i:i+part_size]
        file_path = f"./Output/{name_prefix}_part{i//part_size + 1}.jsonl"
        with open(file_path, "w", encoding="utf-8") as f:
            for doc in part:
                json.dump({"text": doc.page_content}, f)
                f.write("\n")
        print(f"✅ Guardado: {file_path} ({len(part)} chunks)")

# Guardar resultados de ambos splitters
save_chunks_in_parts(char_chunks, "chunks_char_1000")
save_chunks_in_parts(recursive_chunks, "chunks_recursive_1000")


✅ Guardado: ./Output/chunks_char_1000_part1.jsonl (50000 chunks)
✅ Guardado: ./Output/chunks_char_1000_part2.jsonl (50000 chunks)
✅ Guardado: ./Output/chunks_char_1000_part3.jsonl (50000 chunks)
✅ Guardado: ./Output/chunks_char_1000_part4.jsonl (50000 chunks)
✅ Guardado: ./Output/chunks_char_1000_part5.jsonl (21652 chunks)
✅ Guardado: ./Output/chunks_recursive_1000_part1.jsonl (50000 chunks)
✅ Guardado: ./Output/chunks_recursive_1000_part2.jsonl (50000 chunks)
✅ Guardado: ./Output/chunks_recursive_1000_part3.jsonl (50000 chunks)
✅ Guardado: ./Output/chunks_recursive_1000_part4.jsonl (50000 chunks)
✅ Guardado: ./Output/chunks_recursive_1000_part5.jsonl (50000 chunks)
✅ Guardado: ./Output/chunks_recursive_1000_part6.jsonl (50000 chunks)
✅ Guardado: ./Output/chunks_recursive_1000_part7.jsonl (50000 chunks)
✅ Guardado: ./Output/chunks_recursive_1000_part8.jsonl (50000 chunks)
✅ Guardado: ./Output/chunks_recursive_1000_part9.jsonl (50000 chunks)
✅ Guardado: ./Output/chunks_recursive_1000_pa

In [7]:
# Elegimos un modelo rápido, gratuito y potente para retrieval
model = SentenceTransformer("all-MiniLM-L6-v2")

print("✅ Modelo de embeddings cargado correctamente.")

✅ Modelo de embeddings cargado correctamente.


In [4]:
# ⚙️ Función para procesar un archivo de chunks y generar embeddings

def generar_embeddings_desde_jsonl(input_path):
    """
    Carga los chunks desde un archivo .jsonl,
    genera embeddings usando el modelo cargado,
    y devuelve una lista de diccionarios con texto + embedding.
    """
    resultados = []
    with open(input_path, "r", encoding="utf-8") as f:
        for line in f:
            obj = json.loads(line)
            texto = obj.get("text", "").strip()
            if texto:  # Evita procesar líneas vacías
                vector = model.encode(texto).tolist()
                resultados.append({
                    "text": texto,
                    "embedding": vector
                })
    print(f"📦 Embeddings generados desde: {input_path} → {len(resultados)} chunks")
    return resultados


In [5]:
# 💾 Guardado de embeddings en partes (.jsonl)

def guardar_embeddings_por_partes(embeddings, name_prefix, part_size=50000):
    os.makedirs("./Embeddings", exist_ok=True)
    for i in range(0, len(embeddings), part_size):
        parte = embeddings[i:i + part_size]
        ruta = f"./Embeddings/{name_prefix}_part{i//part_size + 1}.jsonl"
        with open(ruta, "w", encoding="utf-8") as f:
            for item in parte:
                json.dump(item, f)
                f.write("\n")
        print(f"✅ Guardado: {ruta} ({len(parte)} vectores)")


In [ ]:
# 🧱 Procesar chunks_recursive_1000_* desde os.listdir()

# Crear carpeta Embeddings si no existe
os.makedirs("./Embeddings", exist_ok=True)

# Buscar todos los archivos en /Output que empiecen con chunks_recursive_1000_part
archivos = [
    f for f in os.listdir("./Output") 
    if f.startswith("chunks_recursive_1000_part") and f.endswith(".jsonl")
]

# Ordenarlos naturalmente por número de parte
archivos_ordenados = sorted(
    archivos,
    key=lambda x: int(x.replace("chunks_recursive_1000_part", "").replace(".jsonl", ""))
)

# Procesar uno por uno
for archivo in archivos_ordenados:
    ruta_entrada = os.path.join("./Output", archivo)
    
    # Extraer número de parte automáticamente
    nro_parte = archivo.replace("chunks_recursive_1000_part", "").replace(".jsonl", "")
    
    # Ruta de salida
    nombre_salida = f"embeddings_recursive_part{nro_parte}.jsonl"
    ruta_salida = os.path.join("./Embeddings", nombre_salida)
    
    # Saltar si ya existe
    if os.path.exists(ruta_salida):
        print(f"⚠️  Ya existe {ruta_salida}, se omite.")
        continue

    print(f"🚀 Procesando {ruta_entrada} → Parte {nro_parte}")
    
    # Generar embeddings
    embeddings = generar_embeddings_desde_jsonl(ruta_entrada)
    
    # Guardar embeddings
    guardar_embeddings_por_partes(embeddings, f"embeddings_recursive_part{nro_parte}")

    print(f"✅ Listo: Parte {nro_parte}\n")


In [ ]:
# 💠 Procesar chunks_char_1000_* desde os.listdir()


# Crear carpeta Embeddings si no existe
os.makedirs("./Embeddings", exist_ok=True)

# Buscar todos los archivos en /Output que empiecen con chunks_char_1000_part
archivos_char = [
    f for f in os.listdir("./Output") 
    if f.startswith("chunks_char_1000_part") and f.endswith(".jsonl")
]

# Ordenarlos naturalmente por número de parte
archivos_char_ordenados = sorted(
    archivos_char,
    key=lambda x: int(x.replace("chunks_char_1000_part", "").replace(".jsonl", ""))
)

# Procesar uno por uno
for archivo in archivos_char_ordenados:
    ruta_entrada = os.path.join("./Output", archivo)
    
    # Extraer número de parte automáticamente
    nro_parte = archivo.replace("chunks_char_1000_part", "").replace(".jsonl", "")
    
    # Ruta de salida
    nombre_salida = f"embeddings_char_part{nro_parte}.jsonl"
    ruta_salida = os.path.join("./Embeddings", nombre_salida)
    
    # Saltar si ya existe
    if os.path.exists(ruta_salida):
        print(f"⚠️  Ya existe {ruta_salida}, se omite.")
        continue

    print(f"🚀 Procesando {ruta_entrada} → Parte {nro_parte}")
    
    # Generar embeddings
    embeddings = generar_embeddings_desde_jsonl(ruta_entrada)
    
    # Guardar embeddings
    guardar_embeddings_por_partes(embeddings, f"embeddings_char_part{nro_parte}")

    print(f"✅ Listo: Parte {nro_parte}\n")


In [42]:
# 🔧 CONFIGURACIÓN: Elige modelo 'char' o 'recursive'
splitter = "recursive"  # ← Cambiar a "recursive" si corresponde

# 📁 Ruta base REAL (la que venís usando vos)
carpeta = "./Embeddings/Overlap 250/"
output_dir = f"./faiss_index_{splitter}"

# 📂 Crear carpeta de salida si no existe
os.makedirs(output_dir, exist_ok=True)

# 📂 Listar los archivos .jsonl del splitter elegido
archivos = sorted([
    archivo for archivo in os.listdir(carpeta)
    if archivo.startswith(f"embeddings_{splitter}") and archivo.endswith(".jsonl")
])

# ✅ Inicializar listas para textos y embeddings
texts = []
embeddings = []
dimensiones = set()

# 📥 Leer textos y embeddings desde cada archivo .jsonl
for archivo in archivos:
    ruta_completa = os.path.join(carpeta, archivo)
    with open(ruta_completa, "r", encoding="utf-8") as f:
        for linea in f:
            obj = json.loads(linea)
            texts.append(obj["text"])
            embeddings.append(obj["embedding"])
            dimensiones.add(len(obj["embedding"]))

# ⚠️ Validar que todas las dimensiones sean iguales
if len(dimensiones) != 1:
    raise ValueError(f"⚠️ Embeddings con dimensiones distintas detectadas: {dimensiones}")

# 🔄 Convertir embeddings a array NumPy (requisito de FAISS)
embeddings_np = np.array(embeddings).astype("float32")

# 🧠 Crear índice FAISS (FlatL2)
index = faiss.IndexFlatL2(embeddings_np.shape[1])
index.add(embeddings_np)

# 💾 GUARDAR EL ÍNDICE VECTORIAL
faiss.write_index(index, os.path.join(output_dir, f"index_{splitter}.faiss"))

# 💾 GUARDAR LOS TEXTOS ASOCIADOS
with open(os.path.join(output_dir, f"texts_{splitter}.pkl"), "wb") as f:
    pickle.dump(texts, f)

# ✅ Mensaje final de confirmación
print(f"✅ Base FAISS creada con éxito para '{splitter}'")
print(f"📄 Documentos indexados: {len(texts)}")
print(f"📁 Carpeta de salida: {output_dir}")



✅ Base FAISS creada con éxito para 'recursive'
📄 Documentos indexados: 488874
📁 Carpeta de salida: ./faiss_index_recursive


In [ ]:
# 🛠️ CONFIGURACIÓN

INDEX_DIR = "./faiss_index_recursive"     # Ruta al índice FAISS creado
EMBEDDING_MODEL = "all-MiniLM-L6-v2"      # Embeddings originales
TOP_K = 8                                 # Número de pasajes a recuperar
LLM_MODEL = "llama3.1"                    # Modelo local vía Ollama (llama3.1, mistral, llama3, etc.)

 
# 🚀 CARGA DE LA BASE FAISS Y TEXTOS ASOCIADOS

print("🔹 Cargando índice y textos guardados…")
index = faiss.read_index(os.path.join(INDEX_DIR, "index_recursive.faiss"))

with open(os.path.join(INDEX_DIR, "texts_recursive.pkl"), "rb") as f:
    textos = pickle.load(f)

print("📊 Mostrando los primeros 3 textos:")
for i, t in enumerate(textos[:3]):
    print(f"Texto {i+1}:\n{repr(t[:300])}\n")

print(f"✅ Índice cargado con éxito: {index.ntotal:,} documentos.")


# 🧠 EMBEDDINGS PARA CONSULTAS

embedder = SentenceTransformer(EMBEDDING_MODEL)

def embed_query(query: str) -> np.ndarray:
    return np.array(embedder.encode(query)).astype("float32")


# 🔍 RECUPERACIÓN SEMÁNTICA

def retrieve(query: str, k: int = TOP_K) -> list[str]:
    query_vec = embed_query(query).reshape(1, -1)
    _, idx = index.search(query_vec, k)
    return [textos[i] for i in idx[0]]


# 🤖 CONFIGURACIÓN DEL MODELO

llm = Ollama(model=LLM_MODEL, temperature=0.7, num_predict=512)

prompt_template = PromptTemplate.from_template(
    textwrap.dedent("""\
    Sos un experto en farmacología clínica. Basado exclusivamente en el contexto provisto, brindá una respuesta completa y detallada que responda específicamente a la siguiente pregunta.
    Si hay información clínica relevante (mecanismo, precauciones, embarazo, interacciones, etc.), incluíla.
    No infieras nada que no esté presente explícitamente.

    CONTEXTO:
    {context}

    PREGUNTA:
    {question}

    RESPUESTA BASADA ÚNICAMENTE EN EL CONTEXTO:
    """)
)



# 🔁 FUNCIÓN RAG

def respuesta_rag(query: str) -> str:
    contexto = "\n\n".join(retrieve(query))
    prompt = prompt_template.format(context=contexto, question=query)
    respuesta = llm.invoke(prompt)
    return respuesta


# ✅ CONSULTA MANUAL DESDE NOTEBOOK

query = "What are the side effects of tacrolimus"


if not query.strip():
    print("❗ Pregunta vacía. Intenta escribir algo.")
else:
    print("\n📚 Pasajes recuperados (TOP_K):")
    for idx, txt in enumerate(retrieve(query), 1):
        print(f"\n• Pasaje {idx}:\n{textwrap.shorten(txt, 350)}")

    print("\n🤖 Respuesta RAG (modelo local):")
    print(respuesta_rag(query))


🔹 Cargando índice y textos guardados…
📊 Mostrando los primeros 3 textos:
Texto 1:
'Professional Monographs Browse medications by letter: Show a list of drugs beginning with the first two letters: Aa Ab Ac Ad Ae Af Ag Ah Ai Aj Ak Al Am An Ao Ap Aq Ar As At Au Av Aw Ax Ay Az 0-9'

Texto 2:
'A/B Otic Uses for A/B Otic: Antipyrine and benzocaine combination is used in the ear to help relieve the pain, swelling, and congestion of some ear infections. It will not cure the infection itself. An antibiotic will be needed to treat the infection. This medicine is also used to soften earwax so t'

Texto 3:
'A/B Otic Before using A/B Otic: In deciding to use a medicine, the risks of taking the medicine must be weighed against the good it will do. This is a decision you and your doctor will make. For this medicine, the following should be considered: Allergies Tell your doctor if you have ever had any un'

✅ Índice cargado con éxito: 488,874 documentos.

📚 Pasajes recuperados (TOP_K):

• Pasaje 1:
T